In [ ]:
"""
Super-7 Vertex AI Extractor (Final Merged Version)

- Uses Vertex AI Gemini with Google Search grounding
- Extracts Super-7 fields + metadata for each company
- 3-attempt retry engine with different search strategies
- Early stop when confidence is high
- LLM-powered dynamic URL classification (gov/registry/official/directory/etc.)
- LLM-generated url_metadata and field_metadata with trust_assessment
- Second-pass enhanced search using high-confidence fields
- Scoring uses: base_confidence × source_type_weight × url_quality × trust_assessment
- process_company_row(...) for single company
- process_csv_print_only(...) for CSV batch (prints JSON for debugging)
"""

import json
import re
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple

import pandas as pd
from google import genai
from google.genai import types


# ==================== CONFIG ====================

PROJECT_ID = "YOUR_GCP_PROJECT_ID"  # <-- change this
LOCATION = "us-east4"
MODEL_NAME = "gemini-2.0-flash"

MAX_RETRIES = 3
EARLY_STOP_THRESHOLD = 0.92  # early-stop if overall score >= this

# Expected CSV columns (you can have more; these are the ones we use)
DEFAULT_INPUT_COLS = [
    "company_name",
    "country",
    "state_province",
    "city",
    "street_address",
    "postal_code",
    "phone_number",
    "additional_info",
]

# Super-7-style fields we score
SUPER7_FIELDS = [
    "company_name",
    "trade_style_name",
    "country",
    "street_address",
    "postal_code",
    "city",
    "state_province",
    "website",
    "phone_number",
]

# Source-type weights for scoring
SOURCE_TYPE_WEIGHTS = {
    "government_registry": 1.0,
    "official_website": 0.9,
    "business_directory": 0.75,
    "social_media": 0.7,
    "high_quality_industry_source": 0.75,
    "other": 0.6,
}

# TLD-based heuristics for URL quality
TLD_WEIGHTS = {
    ".gov": 1.0,
    ".gov.in": 1.0,
    ".gov.uk": 1.0,
    ".edu": 0.9,
    ".com": 0.85,
    ".org": 0.82,
    ".net": 0.8,
    ".co": 0.78,
    ".io": 0.75,
}


@dataclass
class AttemptResult:
    attempt_index: int
    strategy_name: str
    raw_object: Dict[str, Any]
    overall_score: float


# ==================== CLIENT ====================

def init_gemini_client() -> genai.Client:
    """
    Initialize Gemini client with Vertex AI.
    Make sure you have configured ADC (gcloud auth application-default login).
    """
    return genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)


# ==================== STRING / NAME VARIANT HELPERS ====================

CORP_SUFFIXES = [
    "inc", "inc.", "llc", "l.l.c", "corp", "corp.", "corporation",
    "ltd", "ltd.", "limited", "pvt", "pvt.", "pvt ltd", "pty", "pty ltd",
    "gmbh", "ag", "sa", "s.a.", "bv", "srl", "s.r.l.",
]


def normalize_whitespace(s: str) -> str:
    return re.sub(r"\s+", " ", str(s)).strip()


def strip_corp_suffix(name: str) -> str:
    tokens = name.split()
    lowered = [t.lower().strip(",.") for t in tokens]
    end = len(tokens)
    while end > 0 and lowered[end - 1] in CORP_SUFFIXES:
        end -= 1
    return normalize_whitespace(" ".join(tokens[:end]))


def generate_name_variants(company_name: str) -> List[str]:
    """
    Generate multiple useful variants of the company name for search.
    Ensures all variants still contain the core token of the name.
    """
    if not company_name:
        return []

    base = normalize_whitespace(company_name)
    no_suffix = strip_corp_suffix(base)

    variants = set()
    variants.add(base)
    variants.add(no_suffix)

    tokens = no_suffix.split()
    if not tokens:
        return [base]

    # core token
    core = tokens[0]

    if len(tokens) >= 2:
        variants.add(" ".join(tokens[:2]))
    if len(tokens) >= 3:
        variants.add(" ".join(tokens[:3]))

    variants.add(core)
    variants.add(f"{core} llc")
    variants.add(f"{core} ltd")
    variants.add(f"{core} corp")

    cleaned = []
    seen = set()
    for v in variants:
        v2 = normalize_whitespace(v)
        low = v2.lower()
        if v2 and low not in seen:
            cleaned.append(v2)
            seen.add(low)

    # limit to keep prompt compact
    return cleaned[:12]


# ==================== CONTEXT & STRATEGIES ====================

def build_company_context(row_dict: Dict[str, Any]) -> str:
    """
    Build a small context block using CSV fields.
    """
    lines = []
    for col in DEFAULT_INPUT_COLS:
        v = row_dict.get(col)
        if v is not None and str(v).strip():
            lines.append(f"{col}: {v}")
    return "\n".join(lines)


def build_search_strategies(
    row_dict: Dict[str, Any],
    variants: List[str],
) -> List[Tuple[str, str]]:
    """
    Build different textual search strategies (used by the LLM).
    """
    company_name = row_dict.get("company_name", "")
    country = row_dict.get("country", "")
    city = row_dict.get("city", "")
    state = row_dict.get("state_province", "")
    postal = row_dict.get("postal_code", "")

    variants_str = ", ".join(f'"{v}"' for v in variants)
    base_hint = f'Input company name: "{company_name}"'

    strategies: List[Tuple[str, str]] = []

    # Strategy 1: Gov + official
    s1 = f"""
{base_hint}
Use these query patterns heavily:
- Full & near-full names + "{country}" and the location "{city} {state} {postal}".
- Words like "business registry", "official registry", "secretary of state",
  "MCA", "ROC", "Companies House", "ASIC" depending on country.

Prefer:
1) Government or official business registries.
2) Official corporate websites.
Use name variants: {variants_str}
"""
    strategies.append(("gov_official_strategy", s1.strip()))

    # Strategy 2: Address-boosted disambiguation
    s2 = f"""
{base_hint}
Use location as a strong filter to disambiguate companies with similar names:
- Combine "city", "state_province", "postal_code", "{country}" with the name variants.
- Focus on queries like: "<variant>" "<city>" "<state>" "<postal>" "<country>".
Use variants: {variants_str}
"""
    strategies.append(("address_geo_strategy", s2.strip()))

    # Strategy 3: Directory + social fallback
    s3 = f"""
{base_hint}
If registry / official is unclear:
- Search business directories such as LinkedIn, Bloomberg, Crunchbase,
  D&B, Manta, Yelp, BBB, YellowPages, OpenCorporates, industry registries.
- Also consider company pages on social media if structured business info exists.
Use variants: {variants_str}
"""
    strategies.append(("directory_fallback_strategy", s3.strip()))

    return strategies


# ==================== PROMPTS ====================

def build_super7_prompt(
    row_dict: Dict[str, Any],
    strategy_name: str,
    strategy_text: str,
    variants: List[str],
) -> str:
    """
    Main extraction prompt for a single attempt with a given strategy.
    Includes instructions for dynamic URL classification & metadata.
    """
    company_context = build_company_context(row_dict)
    variants_str = ", ".join(f'"{v}"' for v in variants)

    prompt = f"""
You are a highly reliable company-information extractor that uses the Google Search tool.
Your task is to:
1) Use the search strategy,
2) Inspect all grounded URLs,
3) Dynamically classify and score them,
4) Extract Super-7 company fields with detailed metadata.

########################
# COMPANY CONTEXT INPUT
########################
{company_context}

########################
# NAME VARIANTS FOR SEARCH
########################
Use all these name variants in your searches.
Every query MUST still contain the core token from the original name:
{variants_str}

########################
# SEARCH STRATEGY SELECTED ({strategy_name})
########################
{strategy_text}

########################
# URL CLASSIFICATION & METADATA
########################
For EVERY URL you observe via the Google Search tool:
- Examine domain, path, snippet, page title, page content.
- Dynamically classify source_type as one of:
    "government_registry",
    "official_website",
    "business_directory",
    "social_media",
    "high_quality_industry_source",
    "other"
- Estimate trust_assessment (0.0 to 1.0).
- Write a short comment justifying why this URL is classified that way
  (e.g., "official U.S. Secretary of State registry", "corporate about page",
  "generic blog post with no structured company info", etc.).

The classification MUST NOT rely only on hard-coded domain lists:
you must reason from the data, naming patterns, registry words, and actual content.

########################
# FIELDS TO EXTRACT (Super-7 + extra)
########################
Use the BEST subset of URLs to extract:

- company_name          : official legal or main operating name
- trade_style_name      : trading / doing-business-as name (if any)
- country
- street_address
- postal_code
- city
- state_province
- website               : full URL of official corporate site
- phone_number          : primary business phone
- line_of_business      : 1–3 sentence description of what the company does
- reference_url         : up to 5 KEY URLs used in final extraction

For EACH of these fields, provide:
- source_url
- source_type
- trust_assessment (0.0–1.0)
- comment  (short justification)

########################
# URL METADATA
########################
Also provide:
"url_metadata": a list of ALL URLs you found relevant, with:
    {{
      "url": "...",
      "source_type": "...",
      "trust_assessment": 0.0,
      "comment": "..."
    }}

########################
# CONFIDENCE
########################
Provide:
- base_model_confidence: 0.0–1.0 describing how confident you are in the overall object.

########################
# OUTPUT FORMAT (STRICT JSON)
########################
Return EXACTLY a single JSON array with ONE object:

[
  {{
    "company_name": "...",
    "trade_style_name": "...",
    "country": "...",
    "street_address": "...",
    "postal_code": "...",
    "city": "...",
    "state_province": "...",
    "website": "...",
    "phone_number": "...",
    "line_of_business": "...",
    "reference_url": ["...", "..."],
    "url_metadata": [
       {{
          "url": "...",
          "source_type": "government_registry | official_website | business_directory | social_media | high_quality_industry_source | other",
          "trust_assessment": 0.0,
          "comment": "..."
       }}
    ],
    "field_metadata": {{
      "company_name": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }},
      "trade_style_name": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }},
      "country": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }},
      "street_address": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }},
      "postal_code": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }},
      "city": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }},
      "state_province": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }},
      "website": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }},
      "phone_number": {{
        "source_url": "...",
        "source_type": "...",
        "trust_assessment": 0.0,
        "comment": "..."
      }}
    }},
    "base_model_confidence": 0.0
  }}
]

Rules:
- JSON ONLY. No markdown, no extra commentary.
- If a field is not found, set it to null but still provide metadata with "not found" comment.
"""
    return prompt.strip()


# ==================== MODEL CALLS ====================

def clean_json_from_text(raw_text: str) -> str:
    """
    Remove markdown fences and stray text, try to isolate JSON.
    """
    text = raw_text.strip()
    text = re.sub(r"^```json", "", text, flags=re.IGNORECASE).strip()
    text = re.sub(r"^```", "", text).strip()
    if "```" in text:
        text = text.split("```", 1)[0].strip()
    return text


def call_gemini_with_grounding(
    client: genai.Client,
    prompt: str,
) -> Optional[Dict[str, Any]]:
    """
    Call Gemini with Google Search grounding and parse JSON.
    """
    config = types.GenerateContentConfig(
        temperature=0.0,
        tools=[types.Tool(google_search=types.GoogleSearch())],
    )
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt,
            config=config,
        )
        raw_text = response.text or ""
        cleaned = clean_json_from_text(raw_text)
        data = json.loads(cleaned)
        if isinstance(data, list) and data:
            return data[0]
        elif isinstance(data, dict):
            return data
        else:
            print("⚠ Unexpected JSON structure:", type(data))
            return None
    except Exception as e:
        print("⚠ Error in call_gemini_with_grounding:", e)
        return None


# ==================== URL + FIELD SCORING ====================

def extract_domain(url: Optional[str]) -> str:
    if not url:
        return ""
    m = re.search(r"https?://([^/]+)", url)
    return m.group(1).lower() if m else ""


def url_quality(url: Optional[str]) -> float:
    """
    Heuristic URL-quality score using TLD and https.
    """
    if not url:
        return 0.6
    score = 0.7
    domain = extract_domain(url)
    for tld, w in TLD_WEIGHTS.items():
        if domain.endswith(tld):
            score = max(score, w)
    if url.startswith("https://"):
        score += 0.05
    return max(0.4, min(score, 1.0))


def compute_field_score(field_name: str, obj: Dict[str, Any]) -> float:
    """
    Compute score for a single field using:
    - base_model_confidence
    - LLM-provided source_type
    - trust_assessment
    - URL quality
    """
    field_meta = (obj.get("field_metadata") or {}).get(field_name) or {}
    src_type = field_meta.get("source_type", "other")
    trust_assessment = float(field_meta.get("trust_assessment") or 0.3)
    src_url = field_meta.get("source_url")

    base_conf = float(obj.get("base_model_confidence") or 0.0)
    src_weight = SOURCE_TYPE_WEIGHTS.get(src_type, SOURCE_TYPE_WEIGHTS["other"])
    url_w = url_quality(src_url)

    dynamic_factor = 0.5 + 0.5 * trust_assessment  # 0.5–1.0
    score = base_conf * src_weight * url_w * dynamic_factor
    return round(score, 4)


def attach_scores(obj: Dict[str, Any]) -> Dict[str, Any]:
    """
    Attach per-field scores and an overall_confidence_score.
    """
    field_scores: Dict[str, float] = {}
    for f in SUPER7_FIELDS:
        field_scores[f] = compute_field_score(f, obj)

    if field_scores:
        overall = sum(field_scores.values()) / len(field_scores)
    else:
        overall = 0.0

    obj["field_scores"] = field_scores
    obj["overall_confidence_score"] = round(overall, 4)
    return obj


# ==================== SECOND-PASS ENHANCED SEARCH ====================

def run_second_pass_search(
    client: genai.Client,
    primary_obj: Dict[str, Any],
    row_dict: Dict[str, Any],
    variants: List[str],
) -> Optional[Dict[str, Any]]:
    """
    SECOND PASS:
    If any Super-7 field has good confidence (>= 0.75),
    build more precise queries combining the field value + company name,
    then run a second call to refine and possibly improve the result.
    """
    field_scores = primary_obj.get("field_scores") or {}
    high_conf_fields = [f for f, sc in field_scores.items() if sc >= 0.75]

    if not high_conf_fields:
        return None

    cname = primary_obj.get("company_name") or row_dict.get("company_name", "")
    queries = []
    for f in high_conf_fields:
        val = primary_obj.get(f)
        if not val:
            continue
        v_str = normalize_whitespace(val)
        queries.append(f'"{cname}" "{v_str}"')
        queries.append(f'"{cname}" "{v_str}" address')
        queries.append(f'"{cname}" "{v_str}" phone')
        queries.append(f'"{cname}" "{v_str}" registered office')

    if not queries:
        return None

    query_block = "\n".join(f"- {q}" for q in queries)

    second_pass_prompt = f"""
You are performing a SECOND-PASS REFINEMENT for company information.

We already have a provisional object with some high-confidence fields:
{high_conf_fields}

Use these refined queries as a strong hint to Google Search:
{query_block}

Task:
1. Use Google Search tool with these refined queries.
2. Re-check official website, address, and phone, and registry info.
3. If you find more accurate or consistent info, update the fields accordingly.
4. Return the SAME JSON STRUCTURE as before:
   - Super-7 fields
   - url_metadata
   - field_metadata
   - base_model_confidence
5. Your goal is to IMPROVE or CONFIRM the original object.

Return STRICT JSON with a single-element array as before.
"""
    obj = call_gemini_with_grounding(client, second_pass_prompt)
    if not obj:
        return None
    obj = attach_scores(obj)
    return obj


# ==================== RETRY ENGINE ====================

def run_single_attempt(
    client: genai.Client,
    attempt_index: int,
    strategy: Tuple[str, str],
    row_dict: Dict[str, Any],
    variants: List[str],
) -> Optional[AttemptResult]:
    """
    Run a single attempt with a given strategy.
    """
    strat_name, strat_text = strategy
    print(f"\n▶ Attempt {attempt_index + 1}: {strat_name}")
    prompt = build_super7_prompt(row_dict, strat_name, strat_text, variants)
    obj = call_gemini_with_grounding(client, prompt)
    if not obj:
        print("  ⚠ Attempt failed (no valid JSON).")
        return None

    obj = attach_scores(obj)
    overall = obj.get("overall_confidence_score", 0.0)
    print(f"  ✔ Attempt overall score: {overall}")
    return AttemptResult(
        attempt_index=attempt_index,
        strategy_name=strat_name,
        raw_object=obj,
        overall_score=overall,
    )


def process_company_row(
    client: genai.Client,
    row: pd.Series,
) -> Dict[str, Any]:
    """
    Full pipeline for a single company:
    - generate name variants
    - build 3 strategies
    - retry up to MAX_RETRIES with early stop
    - pick best attempt
    - run second-pass enhanced search
    - return final object
    """
    row_dict = row.to_dict()
    company_name = row_dict.get("company_name", "")
    print("=" * 80)
    print(f"Processing company: {company_name}")

    variants = generate_name_variants(company_name)
    strategies = build_search_strategies(row_dict, variants)

    attempts: List[AttemptResult] = []

    print(f"=== RETRY ENGINE START (max {MAX_RETRIES} attempts, early-stop at {EARLY_STOP_THRESHOLD}) ===")
    for i in range(min(MAX_RETRIES, len(strategies))):
        print(f"\n=== Running Attempt {i + 1}/{MAX_RETRIES} ===")
        res = run_single_attempt(client, i, strategies[i], row_dict, variants)
        if res is None:
            continue
        attempts.append(res)
        if res.overall_score >= EARLY_STOP_THRESHOLD:
            print(f"🚀 Early stop triggered (score {res.overall_score} >= {EARLY_STOP_THRESHOLD})")
            break

    if not attempts:
        print("❌ All attempts failed for this company.")
        return {
            "input": row_dict,
            "error": "all_attempts_failed",
        }

    # pick best attempt
    best = max(attempts, key=lambda a: a.overall_score)
    primary = best.raw_object
    print(
        f"✅ Primary winner: Attempt #{best.attempt_index + 1} "
        f"({best.strategy_name}) with score {best.overall_score}"
    )

    # SECOND PASS
    second = run_second_pass_search(client, primary, row_dict, variants)
    if second:
        s_score = second.get("overall_confidence_score", 0.0)
        p_score = primary.get("overall_confidence_score", 0.0)
        if s_score > p_score:
            print(f"🔁 Second-pass improved score from {p_score} → {s_score}. Using refined object.")
            final_obj = second
        else:
            print(f"ℹ Second-pass score {s_score} not better than primary {p_score}. Keeping primary.")
            final_obj = primary
    else:
        print("ℹ Second-pass not executed or returned no better result. Keeping primary.")
        final_obj = primary

    # attach some debug info
    final_obj["best_attempt_index"] = best.attempt_index
    final_obj["best_strategy_name"] = best.strategy_name
    final_obj["input"] = row_dict

    return final_obj


# ==================== CSV PIPELINE (PRINT-ONLY) ====================

def process_csv_print_only(input_csv_path: str) -> List[Dict[str, Any]]:
    """
    Process an input CSV and print JSON results for each row.
    No files are written; everything is printed for debugging.
    """
    df = pd.read_csv(input_csv_path)
    client = init_gemini_client()

    results: List[Dict[str, Any]] = []
    for idx, row in df.iterrows():
        result = process_company_row(client, row)
        results.append(result)

        print("-" * 80)
        print(f"Result for row {idx} (company: {row.get('company_name', '')})")
        print(json.dumps(result, indent=2, ensure_ascii=False))
        print("-" * 80)

    return results


# ==================== CLI ENTRY (OPTIONAL) ====================

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(
        description="Super-7 Vertex AI Extractor (print-only debug mode)."
    )
    parser.add_argument(
        "--input_csv",
        required=True,
        help="Path to input CSV file with company_name, country, etc.",
    )
    args = parser.parse_args()

    process_csv_print_only(args.input_csv)


In [ ]:
from super7_vertex import init_gemini_client, process_company_row
import pandas as pd, json

client = init_gemini_client()

sample_row = pd.Series({
    "company_name": "ABC Global Solutions LLC",
    "country": "United States",
    "state_province": "California",
    "city": "Los Angeles",
    "postal_code": "",
    "street_address": "",
    "phone_number": "",
    "additional_info": ""
})

result = process_company_row(client, sample_row)
print(json.dumps(result, indent=2, ensure_ascii=False))
